In [1]:
from pyspark.sql import SparkSession
import os
import sys
from pyspark.sql.functions import *

import findspark
findspark.init('C:/Spark/spark-3.4.2-bin-hadoop3/')

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


spark = SparkSession.builder.config("spark.driver.memory", "10g").master('local[*]').appName('lab_spark')\
    .config('spark.jars.packages', 'org.apache.spark:spark-avro_2.12:3.4.2')\
    .config('spark.jars', 'C:\\Spark\\spark-3.4.2-bin-hadoop3\\jars\\postgresql-42.7.3.jar')\
    .getOrCreate()

In [2]:
data = spark.read.format("avro").load("orders_avro")

In [3]:
data.show()

+-------+--------------------+---------+-----------+--------+--------+------------+-------+----------+-----------+
|country|       facility_name|      lat|        lng|order_id|    time|        food|  price|      date|       city|
+-------+--------------------+---------+-----------+--------+--------+------------+-------+----------+-----------+
| Russia|         Супра Меоре|43.101581|131.9051689|   73208|01:02:09|     Tom yum| 346.23|2025-07-03|Vladivostok|
| Russia|         Супра Меоре|43.101581|131.9051689|   59817|01:19:49|     Tom yum| 346.23|2025-07-03|Vladivostok|
| Russia|         Супра Меоре|43.101581|131.9051689|   59817|01:19:49|       Pasta| 329.13|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   66084|04:31:58|         Pie| 482.98|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   66084|04:31:58|         Pie| 482.98|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   28648|05:20:32|   Hamburg

In [4]:
dim_country = data.select("country").distinct().\
                withColumn('country_id', monotonically_increasing_id())

dim_facility = data.select("facility_name", "lat", "lng", "city").distinct().\
               withColumn('rest_id', monotonically_increasing_id())

dim_menu = data.select("food",'price').distinct().\
            withColumn('food_id', monotonically_increasing_id())

dim_order = data.select("date",'time').distinct().\
            withColumn('order_id', monotonically_increasing_id())

fact_table = data.select("country", "facility_name", "food", "date", "price")

In [5]:
kwargs = {
    'properties':{
    "user": "postgres",
    "password": "12345",
    "driver": "org.postgresql.Driver"},
    'mode':'append',
    'url':'jdbc:postgresql://localhost:5432/task4',
}



dim_country.write.jdbc(table="country",**kwargs )
dim_facility.write.jdbc(table="facility",**kwargs)
dim_menu.write.jdbc(table="menus",**kwargs)
dim_order.write.jdbc(table="orders",**kwargs)
fact_table.write.jdbc( table="fact_table",**kwargs)